In [74]:
%reset -f

import os
import pandas as pd
import matplotlib.pyplot as plt
import bz2
import ujson
import plotly.express as px
import kaleido


# # Fußverkehrsaufkommen - OTC07
# directory = "Z:/04_Daten/GroundThruth/Fussverkehrausfkommen/OTC07_23-09-19_17-00-00/"
# IOU_files = ['OTCamera07_FR20_2023-09-19_17-00-00_IOU.counts_15min.csv']
# EIOU_files = ['OTCamera07_FR20_2023-09-19_17-00-00_EIOU.counts_15min.csv']
# GT_files = ['OTCamera07_FR20_2023-09-19_17-00-00.otgtevents']
# GTbool = True

# # Fußverkehrsaufkommen - OTC13
directory = "Z:/04_Daten/GroundThruth/Fussverkehrausfkommen/OTC13_23-10-22_17-00-00/"
IOU_files = ['OTCamera13_FR20_2023-10-22_17-00-00_IOU.counts_15min.csv']
EIOU_files = ['OTCamera13_FR20_2023-10-22_17-00-00_EIOU.counts_15min.csv']
GT_files = ['OTCamera13_FR20_2023-10-22_17-00-00.otgtevents']
GTbool = True


# # Leipzig - OTC07
# directory = "Z:/04_Daten/GroundThruth/Leipzig/OTC07/"
# IOU_files = ['OTCamera07_FR20_2022-10-31_16-30-00_IOU.counts_15min.csv']
# EIOU_files = ['OTCamera07_FR20_2022-10-31_16-30-00_EIOU.counts_15min.csv']
# GT_files = ['OTCamera07_FR20_2022-10-31_16-30-00.otgtevents']
# GTbool = True

# # Leipzig - OTC19
# directory = "Z:/04_Daten/GroundThruth/Leipzig/OTC19/"
# IOU_files = ['OTCamera19_FR20_2023-05-24_07-00-00_IOU.counts_15min.csv',
#              'OTCamera19_FR20_2023-05-24_07-15-00_IOU.counts_15min.csv']
# EIOU_files = ['OTCamera19_FR20_2023-05-24_07-00-00_EIOU.counts_15min.csv',
#              'OTCamera19_FR20_2023-05-24_07-15-00_EIOU.counts_15min.csv']
# GT_files = ['OTCamera19_FR20_2023-05-24_07-00-00.otgtevents']
# GTbool = True



# # Saarbrücken - OTC15
# directory = "Z:/04_Daten/GroundThruth/Saarbruecken/OTC15/"
# IOU_files = ['OTCamera15_FR20_2022-10-17_16-00-00_IOU.counts_15min.csv']
# EIOU_files = ['OTCamera15_FR20_2022-10-17_16-00-00_EIOU.counts_15min.csv']
# GT_files = ['.otgtevents']
# GTbool = False

# # Saarbrücken - OTC17
# directory = "Z:/04_Daten/GroundThruth/Saarbruecken/OTC17/"
# IOU_files = ['OTCamera17_FR20_2022-10-19_17-15-00_IOU.counts_15min.csv',]
# EIOU_files = ['OTCamera17_FR20_2022-10-19_17-15-00_EIOU.counts_15min.csv',]
# GT_files = ['OTCamera17_FR20_2022-10-19_17-15-00.otgtevents',]
# GTbool = True

# # TUD
# directory = "Z:/04_Daten/GroundThruth/Videos_TUD/"
# IOU_files = ['otcamera-tud03_FR20_2023-10-26_08-00-00_IOU.counts_15min.csv',
#              'otcamera-tud03_FR20_2023-10-26_08-15-00_IOU.counts_15min.csv',
#              'otcamera-tud03_FR20_2023-10-26_08-30-00_IOU.counts_15min.csv',
#              'otcamera-tud03_FR20_2023-10-26_08-45-00_IOU.counts_15min.csv']
# EIOU_files = ['otcamera-tud03_FR20_2023-10-26_08-00-00_EIOU.counts_15min.csv',
#               'otcamera-tud03_FR20_2023-10-26_08-15-00_EIOU.counts_15min.csv',
#               'otcamera-tud03_FR20_2023-10-26_08-30-00_EIOU.counts_15min.csv',
#               'otcamera-tud03_FR20_2023-10-26_08-45-00_EIOU.counts_15min.csv']
# GT_files = ['otcamera-tud03_FR20_2023-10-26_08-00-00.otgtevents',
#             'otcamera-tud03_FR20_2023-10-26_08-15-00.otgtevents',
#             'otcamera-tud03_FR20_2023-10-26_08-30-00.otgtevents',
#             'otcamera-tud03_FR20_2023-10-26_08-45-00.otgtevents']
# GTbool = True


def compare_EIOU_IOU_GT(directory, IOU_files, EIOU_files, GT_files, GT = True):
    df_IOU = pd.DataFrame()
    for file in IOU_files:
        df = pd.read_csv(directory + file)
        df_IOU = pd.concat([df_IOU, df])

    df_EIOU = pd.DataFrame()
    for file in EIOU_files:
        df = pd.read_csv(directory + file)
        df_EIOU = pd.concat([df_EIOU, df])
    del file, df

    df_IOU = df_IOU[['start occurrence time', 'classification', 'flow', 'count']].rename(columns={'count': 'IOU'})
    df_EIOU = df_EIOU[['start occurrence time', 'classification', 'flow', 'count']].rename(columns={'count': 'EIOU'})
    

    COUNTS = pd.merge(df_IOU, df_EIOU, on = ['start occurrence time', 'classification', 'flow'])
    del df_IOU, df_EIOU
    
    if GT == True:
        df_GT = pd.DataFrame()
        for file in GT_files:
            with bz2.open(directory + file, "rt", encoding="UTF-8") as file:
                dictfile = ujson.load(file)
                
                SECTIONS = pd.DataFrame.from_dict(dictfile['sections'])
                EVENTS = pd.DataFrame.from_dict(dictfile['event_list'])
                EVENTS = pd.merge(EVENTS, SECTIONS, left_on='section_id', right_on='id')
                enter_events = EVENTS.groupby('road_user_id').first().rename(columns={'name': 'name_enter'}).reset_index()
                exit_events = EVENTS.groupby('road_user_id').last().rename(columns={'name': 'name_exit'}).reset_index()
                
                exit_events = exit_events[['road_user_id', 'name_exit']]
                
                GTEVENTS = pd.merge(enter_events, exit_events, on = 'road_user_id').rename(columns={'road_user_type': 'classification'})
                del enter_events, exit_events
                
                GTEVENTS['flow'] = GTEVENTS['name_enter'] + " --> " + GTEVENTS['name_exit']
                GTEVENTS['start occurrence time'] = GTEVENTS['video_name'].str[-13:-5]
                GTEVENTS['start occurrence time'] = GTEVENTS['start occurrence time'].str.replace('-',':')
                
                df_GT = pd.concat([df_GT, GTEVENTS])
                
                
        del dictfile, SECTIONS, EVENTS, GTEVENTS,

        GTCOUNTS = df_GT.groupby(['start occurrence time', 'classification', 'flow'])['type'].count().reset_index().rename(columns={'type': 'GT'})
        COUNTS = pd.merge(COUNTS, GTCOUNTS, on = ['start occurrence time', 'classification', 'flow'])
    else:
        print('Keine GT')
        

    # COUNTS = COUNTS[COUNTS['classification'] != 'pedestrian']
    # COUNTS = COUNTS[COUNTS['classification'] != 'other']
    # Klassen zusammenfassen
    COUNTS['classification_2'] = COUNTS['classification'].replace({'car': 'car',
                                                                'car_with_trailer': 'car',
                                                                'delivery_van': 'van',
                                                                'private_van': 'van',
                                                                'delivery_van_with_trailer': 'van',
                                                                'private_van_with_trailer': 'van',
                                                                'truck': 'truck',
                                                                'truck_with_trailer': 'truck',
                                                                'truck_with_semitrailer': 'truck',
                                                                'bus': 'bus',
                                                                'bicyclist': 'bicycle',
                                                                'bicyclist_with_trailer': 'bicycle',
                                                                'motorcyclist': 'motorcycle',                                                                                                                        
                                                                })
    
    


    if GT == True:
        valuevars = ['IOU', 'EIOU', 'GT']
    else:
        valuevars = ['IOU', 'EIOU']
    
    COUNTS_long = pd.melt(COUNTS, id_vars=['classification_2', 'flow'], value_vars=valuevars).rename(columns={'classification_2': 'Verkehrsmittel',
                                                                                                                'variable': 'Modus',
                                                                                                                'value': 'Anzahl'})
    fig = px.bar(COUNTS_long.groupby(['Verkehrsmittel', 'Modus'])[['Anzahl']].sum().reset_index(), 
             x="Verkehrsmittel", 
             y="Anzahl", 
             color='Modus', 
             barmode='group',
             width=1000,
             height=600,
             text_auto=True).update_layout(xaxis={'categoryorder':'total descending'})
    fig.show()
    # try:
    #     fig.write_image(directory + "/COUNTS.png")
    # except:
    #     print('Bild noch offen')
        
    try:
        COUNTS_long.to_csv(directory + "COUNTS_final.csv", sep=';')
    except:
        print('Datei noch offen!')
    
    
    return COUNTS_long

COUNTS = compare_EIOU_IOU_GT(directory, IOU_files, EIOU_files, GT_files, GT = GTbool)



- Visualisierung

In [77]:
COUNTFILES = ["Z:/04_Daten/GroundThruth/Leipzig/OTC07/COUNTS_final.csv",
              "Z:/04_Daten/GroundThruth/Leipzig/OTC19/COUNTS_final.csv",
              "Z:/04_Daten/GroundThruth/Saarbruecken/OTC15/COUNTS_final.csv",
              "Z:/04_Daten/GroundThruth/Saarbruecken/OTC17/COUNTS_final.csv",
              "Z:/04_Daten/GroundThruth/Fussverkehrausfkommen/OTC07_23-09-19_17-00-00/COUNTS_final.csv",
              "Z:/04_Daten/GroundThruth/Fussverkehrausfkommen/OTC13_23-10-22_17-00-00/COUNTS_final_OTC13.csv",
              "Z:/04_Daten/GroundThruth/Videos_TUD/COUNTS_final.csv"
              ]

COUNTS_GES = pd.DataFrame()
for file in COUNTFILES:
    df = pd.read_csv(file, sep=';')
    df['file'] = file
    COUNTS_GES = pd.concat([COUNTS_GES, df])

# COUNTS_GES.to_csv("Z:/04_Daten/GroundThruth/GT_Uebersicht.csv", sep=";")

dings = COUNTS_GES.groupby('file')['Anzahl'].sum().reset_index()

COUNTS_SUM = COUNTS_GES.groupby(['file', 'Modus', 'Verkehrsmittel', 'flow'])['Anzahl'].sum().reset_index()

fig = px.bar(COUNTS_SUM, 
             x="Verkehrsmittel", 
             y="Anzahl", 
             color='Modus', 
             barmode='group',
             width=1000,
             height=600,
             text_auto=True).update_layout(xaxis={'categoryorder':'total descending'})
# fig.show()
